# Driving a Financial Valuation from a Design Scenario

```{warning}
The following is a work in progress. More content will be added as produced.
```

As seen in
[Loading a Design Scenario](../load_design_scenario.ipynb),
a design scenario's object model (a
[knowledge graph](https://en.wikipedia.org/wiki/Knowledge_graph) including all
its spatial and semantic relationships) can be loaded via Speckle from any of
Speckle's supported [connectors](https://speckle.guide/user/connectors.html), so
long as those objects (`Entity`s and `Relationship`s) have been assigned.

In this notebook, we will demonstrate how to use the object model (graph) to
drive a financial valuation of the design scenario. This is done by calculating
cash flows per `Entity`, and then using the `Relationship`s to aggregate those
`Flow`s into appropriate `Stream`s.

## Load the Design Scenario
We will use the same design scenario as in the previous notebook.

In [2]:
import os
from IPython.display import IFrame

import rangekeeper as rk

In [3]:
speckle = rk.api.Speckle(
    host="speckle.xyz",
    token=os.getenv('SPECKLE_TOKEN'))
stream_id = "3f40d86240"
commit_id = speckle.get_latest_commit_id(stream_id)
IFrame("https://speckle.xyz/embed?stream={0}&commit={1}".format(stream_id, commit_id), width='100%', height=800)


 SpeckleClient( server: https://speckle.xyz, authenticated: True )


### Inspect the Model Graph

In [5]:
model = speckle.get_commit(stream_id=stream_id)
speckle_entities = rk.api.Speckle.parse(model['@scenario'])
root_assembly = rk.api.Speckle.to_entity(speckle_entities[0])
develop = root_assembly.develop(
    name='design_scenario',
    type='design_scenario')
develop.plot(
    notebook=True,
    hierarchical_layout=False,
    display=False)

design_scenario.html


```{raw} html
:file: design_scenario.html
```

## Assigning and Aggregating `Flow`s per `Entity`
Now we can organise the two main categories of cash flows (costs and revenues)
according to the `Relationship` types in the design scenario.

In this example, revenue-based cash flows are generated by `Entity`s that have a
 "sellable" (or "leasable") area -- an NSA or NLA measure.

We can query the graph to find them:

In [17]:
entities = [rk.api.Speckle.to_entity(speckle_entity) for speckle_entity in speckle_entities]
revenue_entities = [entity for entity in entities if hasattr(entity.measurements, 'nsa') or hasattr(entity.measurements, 'nla')]
revenue_entities

[]


They are also aggregated by their
spatial containment. Ie, we wish to sequentially sum their revenues